In [188]:
import math, sympy
from fpylll import *

import  random
def rand_prime(bitsize):
    return sympy.randprime(2**(bitsize-1), 2**(bitsize))                        
def base_primes(num_primes):
    primes = list(sympy.primerange(a=2, b=10 ** 5))
    return primes[:num_primes]
def int2bitstring(value, size = None):
    bits = "{:b}".format(value)
    if size:
        bits = '0'*(size-len(bits)) + bits
    return bits    
def M_sizes(matrix):
    res_matrix = IntegerMatrix(matrix)
    nrows = matrix.nrows
    ncols = matrix.ncols
    for i in range(nrows):
        for j in range(ncols):
            res_matrix[i, j] = len(int2bitstring(matrix[i, j]))
    return res_matrix
def equations2reduction(nlin_eqs = 1, dim = 2):
    L = IntegerMatrix(nlin_eqs+dim, dim)
    for i in range(nlin_eqs):
        for j in range(dim):
            L[i, j] = random.randint(0, M)
    for j in range(dim):
        L[nlin_eqs+j, j] = M
    LLL.reduction(L)
    return M_sizes(L)

## Rows count vs reduction

In [189]:
bit_size = 300
M = 2**bit_size

for nlin_eqs in [1,2,3]:
    dim = nlin_eqs + 1
    print(f"num_of equations={nlin_eqs}, num_variables={dim}, \nElement sizes:")
    print(equations2reduction(nlin_eqs = nlin_eqs, dim = dim))

num_of equations=1, num_variables=2, 
Element sizes:
[   1   1 ]
[ 151 150 ]
[ 151 150 ]
num_of equations=2, num_variables=3, 
Element sizes:
[   1   1   1 ]
[   1   1   1 ]
[  99  99 100 ]
[ 101  98  99 ]
[  98 102 102 ]
num_of equations=3, num_variables=4, 
Element sizes:
[  1  1  1  1 ]
[  1  1  1  1 ]
[  1  1  1  1 ]
[ 73 72 74 73 ]
[ 76 69 74 73 ]
[ 73 74 72 76 ]
[ 73 78 74 73 ]


## Univariate Coppersmith (Howgram-Graham variant)
How large $A$ must be to find $x$ assuming:
 $$(xM+A)(yM+B) = N = p.q$$



In [190]:
bit_size = 10
known_bits = 9
p = rand_prime(bit_size)
q = rand_prime(bit_size)
N = p*q
M = 2**(known_bits)
X = 2**(bit_size-known_bits+1)
A = p % M
x_reference = ( p - A)  // M
assert x_reference < X

In [191]:
from sympy import poly, degree, Poly
from sympy.abc import x

fx = poly(x + (A*pow(M, -1, N)) % N)
assert fx.eval(x_reference) % p == 0 

In [192]:
# def construct_polynomials(fx, N, m):
#     polynomials = []
#     for i in range(m+1):
#         for j in range(m+1):
#             for k in range(m+1):
#                 fi = (x**i)*(fx**k)*(N**j)
#                 if (i+k <= m) and (m < j+k):
#                     polynomials.append(fi)
#     return polynomials 

def construct_polynomials(fx, N, m, additional):
    polynomials = []
    for i in range(m):
        gi = (fx**i)*(N**(m-i))
        polynomials.append(gi)
    for j in range(additional):
        gi = (fx**m)*x**j
        polynomials.append(gi)
    return polynomials 

def sympoly_to_list(polynomial, dim):
    r = list(reversed(list(map(int, polynomial.as_list(m)))))
    padding = [0]*(dim - len(r))
    return r + padding

def col_weights(M, X, add_weights = True):
    n, m = M.nrows, M.ncols
    for i in range(n):
        for j in range(m):
            if add_weights:
                M[i, j] *= X**j
            else:
                M[i, j] //= X**j
    return M              
def matrix_from_polynomials(polynomials):
    max_degree = max([degree(polynomial) for polynomial in polynomials]) + 1
    matrix = [sympoly_to_list(polynomial, max_degree) for polynomial in polynomials]
    L  = IntegerMatrix.from_matrix(matrix)
    return L
def polynomials_from_matrix(M):
    return [Poly.from_list(reversed(list(row)), x) for row in M]

m = 2
basis_polynomials = construct_polynomials(fx=fx, N = N, m = m, additional=7)
assert [polynomial.eval(x_reference) % p for polynomial in basis_polynomials] == [0]*len(basis_polynomials)   
# polynomials = construct_polynomials(fx=fx, N = 10, m = m, additional=4)
L  = matrix_from_polynomials(basis_polynomials)
print(M_sizes(L))
L_with_weights = col_weights(L, X, add_weights=True)

[ 38  1  1  1  1  1  1  1 1 ]
[ 38 19  1  1  1  1  1  1 1 ]
[ 37 20  1  1  1  1  1  1 1 ]
[  1 37 20  1  1  1  1  1 1 ]
[  1  1 37 20  1  1  1  1 1 ]
[  1  1  1 37 20  1  1  1 1 ]
[  1  1  1  1 37 20  1  1 1 ]
[  1  1  1  1  1 37 20  1 1 ]
[  1  1  1  1  1  1 37 20 1 ]


In [193]:
LLL.reduction(L_with_weights)
L_removed_weights = col_weights(L_with_weights, X, add_weights=False)
print(L)
g_polynomials = polynomials_from_matrix(L_removed_weights)

for polynomial in g_polynomials:
    assert polynomial.eval(x_reference) % p == 0
    if polynomial.eval(X) < p**m:
        print(polynomial.eval(x_reference))

[    756  -753   36  -37  -4  1  1 0 0 ]
[   -563   424   69   47  21  2  0 0 0 ]
[   1371 -1185 -134  -69  18  0 -1 0 0 ]
[  -1680  1638  166 -125   6 -5  0 0 0 ]
[    -16     0   40   -8 -25 10 -1 0 0 ]
[  -1572  2354 -770  -28  15 -1  2 0 0 ]
[   -478   135  357   -4  -8 -3  0 1 0 ]
[    371    70 -455   16  -2 -1  0 0 1 ]
[ 595231 37063 2774  133  12 -4  0 0 0 ]
0
0
0
0
0
0
0
0
